In [85]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.2f}'.format
import hvplot.pandas
from datetime import datetime, timedelta
import plotly.express as px
import altair as alt
from vega_datasets import data
import pandas_datareader as web

In [86]:
initial_investment = 10000

In [87]:
tickers_df = portfolio_df = pd.DataFrame({"date": pd.date_range(start='2023-01-01', end='2023-05-21', freq='B')})
tickers_df['AAPL'] = np.random.randint(0,100, size=len(tickers_df))
tickers_df['TSLA'] = np.random.randint(0,100, size=len(tickers_df))
tickers_df['AMD'] = np.random.randint(0,100, size=len(tickers_df))
tickers_df.set_index('date', inplace=True)
tickers_df

,AAPL,TSLA,AMD
date,,,
2023-01-02,71,32,93
2023-01-03,46,33,89
2023-01-04,90,8,47
2023-01-05,55,59,90
2023-01-06,59,84,96
...,...,...,...
2023-05-15,63,86,65
2023-05-16,37,94,19
2023-05-17,23,15,39


In [88]:
tickers = ["AAPL", "TSLA", "DIS"]
weights = [(1 / len(tickers))] * len(tickers)

In [89]:
def cumulative_returns(df, weights, investment):
    pct_change = df.pct_change()
    added_weights = pct_change.dot(weights)
    calculation = (1 + added_weights).cumprod()
    profit = (investment * calculation)
    profit_df = pd.DataFrame(profit)
    profit_df.columns = ['profit']
    return profit_df

In [90]:
user_cr = cumulative_returns(tickers_df, weights, initial_investment)
user_cr

,profit
date,
2023-01-02,NaN
2023-01-03,8787.09
2023-01-04,7987.57
2023-01-05,26361.65
2023-01-06,31309.93
...,...
2023-05-15,inf
2023-05-16,inf
2023-05-17,inf


In [91]:
def covariance(df, ticker, market):
    covariance_rolling = df[ticker].rolling(window=21).cov(df[market])
    return covariance_rolling

In [92]:
user_covariance = covariance(tickers_df, 'AAPL', 'TSLA')
user_covariance

date
2023-01-02       NaN
2023-01-03       NaN
2023-01-04       NaN
2023-01-05       NaN
2023-01-06       NaN
               ...  
2023-05-15    -83.67
2023-05-16    -28.25
2023-05-17    -55.18
2023-05-18   -109.30
2023-05-19   -130.69
Length: 100, dtype: float64

In [93]:
def variance(df, market):
    rolling_variance = df[market].rolling(window=21).var()
    return rolling_variance

In [94]:
user_variance = variance(tickers_df, 'TSLA')
user_variance

date
2023-01-02       NaN
2023-01-03       NaN
2023-01-04       NaN
2023-01-05       NaN
2023-01-06       NaN
               ...  
2023-05-15   1087.33
2023-05-16   1168.36
2023-05-17   1127.03
2023-05-18   1091.10
2023-05-19   1106.05
Name: TSLA, Length: 100, dtype: float64

In [95]:
def daily_drawdown(df):
    roll_max = df.cummax()
    roll_min = df.cummin()
    daily_drawdown = round(((roll_max - roll_min) / roll_max)*100, 2)
    return daily_drawdown

In [96]:
user_daily_drawdown = daily_drawdown(tickers_df)
user_daily_drawdown

,AAPL,TSLA,AMD
date,,,
2023-01-02,0.00,0.00,0.00
2023-01-03,35.21,3.03,4.30
2023-01-04,48.89,75.76,49.46
2023-01-05,48.89,86.44,49.46
2023-01-06,48.89,90.48,51.04
...,...,...,...
2023-05-15,98.96,98.99,100.00
2023-05-16,98.96,98.99,100.00
2023-05-17,98.96,98.99,100.00


In [97]:
max_user_drawdown = max(user_daily_drawdown)
max_user_drawdown

'TSLA'

In [98]:
def tracking_error(df, tickers, market):
    trackingerror_i = (df[tickers] - df[market]).rolling(window=21).std()
    trackingerror_df = trackingerror_i.to_frame()
    trackingerror_df = trackingerror_df.dropna()
    trackingerror_df.columns = ['tracking error']
    return trackingerror_df

In [99]:
user_tracking_error = tracking_error(tickers_df, 'AAPL', 'TSLA')
user_tracking_error

,tracking error
date,
2023-01-30,34.74
2023-01-31,35.56
2023-02-01,35.80
2023-02-02,30.28
2023-02-03,30.36
...,...
2023-05-15,43.50
2023-05-16,42.19
2023-05-17,42.14


In [102]:
def beta(covariance, variance):
    user_beta = covariance / variance
    user_beta_df = pd.DataFrame(user_beta)
    user_beta_df.columns = ['beta']
    user_beta_df = user_beta_df.dropna()
    return user_beta_df

In [103]:
user_beta = beta(user_covariance, user_variance)
user_beta

,beta
date,
2023-01-30,0.21
2023-01-31,0.20
2023-02-01,0.22
2023-02-02,0.39
2023-02-03,0.41
...,...
2023-05-15,-0.08
2023-05-16,-0.02
2023-05-17,-0.05


In [ ]:
def sharpe_ratio(df):
    sharpe = (df.mean()*252) / (df.std() * np.sqrt(252))
    return sharpe

In [ ]:
user_sharpe_ratio = sharpe_ratio(portfolio_df)
user_sharpe_ratio

AAPL   27.35
TSLA   25.56
AMD    30.34
dtype: float64

In [ ]:
def return_on_investment(investment, returns):
    cumulative_profit = investment * returns
    return_oi = (cumulative_profit - investment) / investment
    return(return_oi)

In [ ]:
user_roi = return_on_investment(initial_investment, user_cr)
user_roi

,profit
date,
2023-01-02,NaN
2023-01-03,6106.21
2023-01-04,18796.69
2023-01-05,14758.67
2023-01-06,34931.86
...,...
2023-05-15,inf
2023-05-16,inf
2023-05-17,inf


In [ ]:
def standard_deviation(df):
    rolling_std = df.rolling(window = 21).std()
    rolling_std_df = pd.DataFrame(rolling_std)
    rolling_std_df = rolling_std_df.dropna()
    return rolling_std_df

In [ ]:
user_std = standard_deviation(tickers_df)
user_std

,AAPL,TSLA,AMD
date,,,
2023-01-30,24.78,29.34,28.40
2023-01-31,25.22,29.51,27.21
2023-02-01,27.49,28.61,27.26
2023-02-02,27.81,28.06,26.53
2023-02-03,28.40,28.97,25.75
...,...,...,...
2023-05-15,31.02,27.12,23.54
2023-05-16,28.67,26.13,23.67
2023-05-17,28.62,25.29,23.64


In [ ]:
def annual_return(df):
    return (1+df)**.2-1

In [ ]:
user_annual_return = annual_return(user_cr)
user_annual_return

,profit
date,
2023-01-02,NaN
2023-01-03,4.72
2023-01-04,6.16
2023-01-05,5.82
2023-01-06,7.10
...,...
2023-05-15,inf
2023-05-16,inf
2023-05-17,inf


In [ ]:
def portfolio_distribution_chart(tickers, weights):
    chart = px.pie(values=weights, names=tickers, hole=.5)
    return chart

In [ ]:
user_portfolio_distribution = portfolio_distribution_chart(tickers, weights)
user_portfolio_distribution

In [ ]:
def user_tracking_error_chart(df):
    fig = px.line(df, x=df.index, y=df.columns)
    return fig

In [ ]:
user_te_chart = user_tracking_error_chart(user_tracking_error)
user_te_chart

In [ ]:
def user_cumulative_return_chart(df):
    fig = px.line(df, x=df.index, y=df.columns)
    return fig

In [ ]:
user_cr_chart = user_cumulative_return_chart(user_cr)
user_cr_chart